# Zen Garden Design Analysis

## 1. Scrape

To collect our design data, we scrape csszengarden.com for design screenshots and associated styles. With over 200 designs, this should give us a good training set on how apply different styles and techniques.

In [2]:
from data_collection.scraper import scrape_design
import asyncio

async def test_scraper(ids, batch_size=5):
    """
    Asynchronously scrape designs in batches to avoid overwhelming resources.
    
    Args:
        ids (list): List of design IDs to scrape
        batch_size (int): Number of designs to process concurrently
    """
    print(f"Starting scrape of {len(ids)} designs...")
    
    successful = 0
    failed = 0
    
    # Process in batches
    for i in range(0, len(ids), batch_size):
        batch = ids[i:i + batch_size]
        print(f"\nProcessing batch {i//batch_size + 1} ({len(batch)} designs)...")
        
        # Create tasks for current batch
        tasks = [scrape_design(design_id) for design_id in batch]
        
        # Run batch tasks concurrently
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Process batch results
        for design_id, result in zip(batch, results):
            if isinstance(result, Exception):
                print(f"Error scraping design {design_id}: {str(result)}")
                failed += 1
            else:
                print(f"Successfully scraped design {design_id}")
                successful += 1
        
        # Optional: Add delay between batches
        # await asyncio.sleep(1)
    
    print(f"\nScraping complete:")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Total: {len(ids)}")

# Example usage with batch processing:
test_set = [f"{i:03d}" for i in range(1, 222)]
await test_scraper(test_set, batch_size=5)

Starting scrape of 221 designs...

Processing batch 1 (5 designs)...
001: Response status: 200
002: Response status: 200
003: Response status: 200
004: Response status: 200
005: Response status: 200
Successfully scraped design 001
Successfully scraped design 002
Successfully scraped design 003
Successfully scraped design 004
Successfully scraped design 005

Processing batch 2 (5 designs)...
006: Response status: 200
007: Response status: 200
008: Response status: 200
009: Response status: 200
010: Response status: 200
Successfully scraped design 006
Successfully scraped design 007
Successfully scraped design 008
Successfully scraped design 009
Successfully scraped design 010

Processing batch 3 (5 designs)...
011: Response status: 200
012: Response status: 200
013: Response status: 200
014: Response status: 200
015: Response status: 200
Successfully scraped design 011
Successfully scraped design 012
Successfully scraped design 013
Successfully scraped design 014
Successfully scraped de

Now for a valuable additional step, we can extract the name of the title and its author from the CSS comments. This helps us give credit to the creative people whose work we are using and appreciating.

In [1]:
from data_collection.analyze_designs import attribute_designs
await attribute_designs()

Found 221 designs to check
Skipping design 135 - already attributed
Skipping design 132 - already attributed
Skipping design 104 - already attributed
Skipping design 103 - already attributed
Skipping design 168 - already attributed
Skipping design 157 - already attributed
Skipping design 150 - already attributed
Skipping design 159 - already attributed
Skipping design 166 - already attributed
Skipping design 192 - already attributed
Skipping design 195 - already attributed
Skipping design 161 - already attributed
Skipping design 102 - already attributed
Skipping design 105 - already attributed
Skipping design 133 - already attributed
Skipping design 134 - already attributed
Skipping design 160 - already attributed
Skipping design 194 - already attributed
Skipping design 158 - already attributed
Skipping design 193 - already attributed
Skipping design 167 - already attributed
Skipping design 151 - already attributed
Skipping design 169 - already attributed
Skipping design 156 - already 

## 2. Analyze

Now, using the screenshots and styles we downloaded, we analyze the design for characteristics that will be useful for retrieval. Our analyzer can perform a basic analysis and a detailed analysis, which will be used to test and illustrate results on the data set later.

In [1]:
from data_collection.analyze_designs import analyze_screenshot
from pathlib import Path
import asyncio

async def test_analyzer(design_ids, batch_size=5, detailed=True, output_path=None):
    """
    Asynchronously analyze designs in batches.
    
    Args:
        design_ids (list): List of design IDs to analyze
        batch_size (int): Number of designs to process concurrently
        detailed (bool): Whether to use detailed analysis
        output_path (Path): Where to save analysis results
    """
    print(f"Starting analysis of {len(design_ids)} designs...")
    
    successful = 0
    failed = 0
    
    # Process in batches
    for i in range(0, len(design_ids), batch_size):
        batch = design_ids[i:i + batch_size]
        print(f"\nProcessing batch {i//batch_size + 1} ({len(batch)} designs)...")
        
        # Create tasks for current batch
        tasks = [
            analyze_screenshot(
                design_id=design_id,
                design_path=Path(f"scraped_designs/{design_id}"),
                detailed=detailed,
                output_path=output_path
            ) for design_id in batch
        ]
        
        # Run batch tasks concurrently
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Process batch results
        for design_id, result in zip(batch, results):
            if isinstance(result, Exception):
                print(f"Error analyzing design {design_id}: {str(result)}")
                failed += 1
            elif result[1] is not None:  # Check if analysis was successful
                successful += 1
                print(f"\nAnalysis for design {design_id}:")
                if detailed:
                    print(f"Description: {result[1]}")  # summary from description object
                else:
                    print(f"Description: {result[1]}")  # direct description string
                print(f"Categories: {', '.join(result[2])}")
                print(f"Visual Characteristics: {', '.join(result[3])}")
            else:
                print(f"Failed to analyze design {design_id}")
                failed += 1
    
    print(f"\nAnalysis complete:")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Total: {len(design_ids)}")


Now we can run the analysis on a range of our choosing.

In [3]:

# Test with detailed analysis
print("Running detailed analysis...")
analysis_test_set = [f"{i:03d}" for i in range(200, 205)]
await test_analyzer(
    design_ids=analysis_test_set,
    batch_size=5,
    detailed=True,
    output_path=Path("analyses/detailed")
)

# Test with basic analysis
print("\nRunning basic analysis...")
await test_analyzer(
    design_ids=analysis_test_set,
    batch_size=5,
    detailed=False,
    output_path=Path("analyses/basic")
)

Running detailed analysis...
Starting analysis of 5 designs...

Processing batch 1 (5 designs)...
Analyzing design 200...
Analyzing design 201...
Analyzing design 202...
Analyzing design 203...
Analyzing design 204...
Successfully analyzed design 203
Successfully analyzed design 200
Successfully analyzed design 201
Successfully analyzed design 204
Successfully analyzed design 202

Analysis for design 200:
Description: A serene, atmospheric web design featuring a gradient blue backdrop with zen-inspired imagery and structured content areas. The page employs a two-column layout with an asymmetric balance and thoughtful typographic hierarchy.
Categories: Minimalist digital, Zen-inspired web design, Atmospheric interface, Nature-digital fusion, Gradient composition
Visual Characteristics: Monochromatic blue gradient palette, Silhouetted natural elements, Textural contrast between smooth and rough edges, Two-column asymmetric layout, Vertical rhythm with horizontal section breaks

Analysis 